In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from skimage.transform import resize

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
# image generator used in valid_data and test_data
val_idg = keras.preprocessing.image.ImageDataGenerator(rescale = 1/ 255.0, 
                         samplewise_center=True, 
                          samplewise_std_normalization=True
                        )

def check_dicom(filename): 
    # todo
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    print('{} information:'.format(filename))
    print('Modality: {}'.format(ds.Modality))
    print('Body Part Examined: {}'.format(ds.BodyPartExamined))
    print('Patient Age: {}'.format(ds.PatientAge))
    print('Patient Position: {}'.format(ds.PatientPosition)) 
    if ds.Modality != 'DX':
        print('Not a valid input data: Modality is not DX')
        return
    if ds.BodyPartExamined != 'CHEST':
        print('Not a valid input data: Body Part Examined is not CHEST')
        return
    if not (1 <= int(ds.PatientAge) <= 90):
        print('Not a valid input data: patient age is out of range')
        return
    if ds.PatientPosition not in {'AP', 'PA'}:
        print('Not a valid input data: Patient Position is not AP or PA')
        return
    return ds.pixel_array
    
def preprocess_image(img, target_size): 
    # todo
    img = resize(img, (target_size[1], target_size[2]))
    proc_img = np.zeros(target_size)
    # make the input image as target_size
    for i in range(target_size[3]):
        proc_img[0][:,:,i] = img
    proc_img = val_idg.flow(proc_img).next()
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    with open(model_path, 'r') as json_file:
        model_json = json_file.read()
        model = tf.keras.models.model_from_json(model_json)
        model.load_weights(weight_path)
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo    
    labels = ['No Finding', 'Pneumonia']
    score = model.predict(img)[0][0]
    prediction = labels[int(score >= thresh)]
    return prediction, score

In [4]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

#path to saved model
model_path = 'my_model.json'
#path to saved best weights
weight_path = 'xray_class_my_model.best.hdf5'

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16

In [5]:
#loads model
my_model = load_model(model_path, weight_path)
#loads the threshold they chose for model classification 
thresh = 0.5 
    
for i in test_dicoms:
    print('='*80)
    img = np.array([])
    img = check_dicom(i)
    if img is None:
        continue
    img_proc = preprocess_image(img, IMG_SIZE)
    pred, score = predict_image(my_model,img_proc,thresh)
    print('Predicted label: {}, Score: {:.2f}'.format(pred, score))

Load file test1.dcm ...
test1.dcm information:
Modality: DX
Body Part Examined: CHEST
Patient Age: 81
Patient Position: PA
Predicted label: No Finding, Score: 0.42
Load file test2.dcm ...
test2.dcm information:
Modality: DX
Body Part Examined: CHEST
Patient Age: 58
Patient Position: AP
Predicted label: No Finding, Score: 0.44
Load file test3.dcm ...
test3.dcm information:
Modality: DX
Body Part Examined: CHEST
Patient Age: 77
Patient Position: AP
Predicted label: Pneumonia, Score: 0.76
Load file test4.dcm ...
test4.dcm information:
Modality: DX
Body Part Examined: RIBCAGE
Patient Age: 81
Patient Position: PA
Not a valid input data: Body Part Examined is not CHEST
Load file test5.dcm ...
test5.dcm information:
Modality: CT
Body Part Examined: CHEST
Patient Age: 81
Patient Position: PA
Not a valid input data: Modality is not DX
Load file test6.dcm ...
test6.dcm information:
Modality: DX
Body Part Examined: CHEST
Patient Age: 81
Patient Position: XX
Not a valid input data: Patient Positio